In [5]:
# Cell 1: Import Libraries and Custom Modules
import yfinance as yf
import data_utils
import strategies
import backtest
import grid_search  # Import the new grid search module

# Cell 2: Set Parameters and Prepare Data
# Specify the ticker and start date
ticker = "BBCA.JK"
start_date = "2020-01-01"

# Prepare data using a utility function
real_data = data_utils.prepare_data(ticker, start_date)

# Cell 3: Define Parameter Grids for Grid Search
parameter_grids = {
    "LongOnlyMACD": {
        "window_slow": [24, 26, 28],
        "window_fast": [10, 12, 14],
        "window_sign": [8, 9, 10]
    },
    "LongOnlyMomentum": {
        "window": [10, 20, 30]
    },
    "StochasticOscillatorStrategy": {
        "k_window": [10, 14, 20],
        "smooth_k": [2, 3],
        "smooth_d": [2, 3],
        "threshold": [10, 20]
    }
}

# Cell 4: Perform Grid Search and Print Best Results
if real_data is not None:
    best_results = {}
    
    for strategy_name, param_grid in parameter_grids.items():
        print(f"\nPerforming grid search for {strategy_name}...")
        
        # Perform grid search for each strategy
        results = grid_search.grid_search(
            strategy_name=strategy_name,
            param_grid=param_grid,
            data=real_data,
            symbol=ticker
        )
        
        # Store the best performance metrics for each strategy
        best_results[strategy_name] = {
            "best_perf": results["best_perf"],
            "best_outperf": results["best_outperf"],
            "best_params_perf": results["best_params_perf"],
            "best_params_outperf": results["best_params_outperf"]
        }
        
    # Print the final best perf and outperf for each strategy
    for strategy_name, result in best_results.items():
        print(f"\nStrategy: {strategy_name}")
        print(f"Best Performance (perf): {result['best_perf']} with parameters: {result['best_params_perf']}")
        print(f"Best Outperformance (outperf): {result['best_outperf']} with parameters: {result['best_params_outperf']}")
        
        # Run backtest for the best perf configuration
        print(f"\nRunning best configuration for {strategy_name} based on best perf:")
        perf, outperf, result  = backtest.run_backtest(
            strategy_class=strategies.__getattribute__(strategy_name),
            data=real_data,
            symbol=ticker,
            print_results=False,
            **result['best_params_perf']
        )
        print(result)

else:
    print("No data available for backtesting.")


[*********************100%%**********************]  1 of 1 completed



Performing grid search for LongOnlyMACD...

Performing grid search for LongOnlyMomentum...

Performing grid search for StochasticOscillatorStrategy...


C:\Users\Intel X Nvidia RTX\anaconda3\envs\py10\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.




Strategy: LongOnlyMACD
Best Performance (perf): 1.2079734432742477 with parameters: {'window_slow': 26, 'window_fast': 14, 'window_sign': 10}
Best Outperformance (outperf): -8.799847230057749 with parameters: {'window_slow': 28, 'window_fast': 14, 'window_sign': 9}

Running best configuration for LongOnlyMACD based on best perf:
total_duration                5282 days 00:00:00
start_date                   2010-02-19 00:00:00
end_date                     2024-08-06 00:00:00
leverage                                       1
trading_costs                                0.0
equity_initial                              1000
traded_amount                               1000
exposure_time                          48.065059
equity_final                         1207.973443
equity_peak                          1451.307315
return_pct                             20.797344
return_pct_annualized                   1.267594
volatility_pct_annualized              16.750608
buy_and_hold_return            

In [2]:
backtest

<module 'backtest' from 'C:\\Users\\Intel X Nvidia RTX\\ZenTraderML\\ZenTraderML-\\backtest.py'>